# Optimization log

In [1]:
import torch
from torch.utils.benchmark import Compare, Fuzzer, FuzzedParameter, FuzzedTensor, ParameterAlias, Timer

from torch_image_binarization.thresholding import su

The algorithm is based on the NumPy implementation at https://github.com/nopperl/vectorized-image-binarization. In order to improve performance, the code was rewritten in Pytorch. Performance is enabled in two ways: first, GPU acceleration with CUDA. Secondly, the Pytorch code can be automatically converted into efficient Triton kernels using `torch.compile`. Since [`torch.compile` supports NumPy starting with version 2.1](https://pytorch.org/docs/stable/torch.compiler_faq.html#does-numpy-work-with-torch-compile), the NumPy-to-Pytorch conversion possibly could also be skipped. The latest Pytorch version (2.3) is used as earlier versions produced errors.

Since the Otsu implementation of OpenCV used in the NumPy version of the algorithm is incompatible with Pytorch, a Pytorch version of Otsu's method is written. Initially, the [loop-based version of Otsu's method from OpenCV](https://docs.opencv.org/3.4/d7/d4d/tutorial_py_thresholding.html) was converted to Pytorch, but (expectedly) performed poorly (e.g., each iteration of the loop was converted into a Triton kernel). Therefore, a vector-based version inspired by [scikit-image](https://github.com/scikit-image/scikit-image/blob/v0.22.x/skimage/filters/thresholding.py#L312) was implemented in Pytorch.

One easy performance improvement would be to perform calculations in bf16, but this lead to overflows due to the use of `torch.cumsum`.

An integral part of Otsu's method is computing the image histogram. Unfortunately, the `torch.histogram` function is not supported by `torch.compile`. Therefore, a `histogram` function for grayscale images compatible with `torch.compile` was written.

Together with the aforementioned changes, the code was improved to eliminate all graph breaks. Compiling the resulting singular graph with dynamic shapes leads to the Triton kernel in `compiled_kernel.py`.

## Benchmarks
Now, benchmark the performance gains by using `torch.compile`.

In [2]:
seed=123
torch.manual_seed(seed)

In [3]:
def benchmark(stmts, tensors_fixed=True, seed=seed):
    if tensors_fixed:
        tensor_generator = ((torch.rand(1, 2 ** i, 2 ** i, device="cuda")) for i in range(8, 13))
    else:
        img_fuzzer = Fuzzer(
            parameters = [
                FuzzedParameter("h", minval=256, maxval=8096, distribution='loguniform'),
                FuzzedParameter("w", minval=256, maxval=8096, distribution='loguniform'),
            ],
            tensors = [
                FuzzedTensor("img", size=(1, "h", "w"), probability_contiguous=1, cuda=True)
            ],
            seed=0,
        )
        tensor_generator = img_fuzzer.take(10)

    measurements = []
    for item in tensor_generator:
        tensors = item[0]
        for stmt in stmts:
            measurement = Timer(
                stmt=stmt,
                setup="from __main__ import su",
                globals=tensors,
                label="su",
                description="runtime",
            ).blocked_autorange(min_run_time=1)
            measurements.append(measurement)
    compare = Compare(measurements)
    return compare

In [4]:
stmts = [
    "su(img)",
    "torch.compile(su)(img)",
    "torch.compile(su, mode='reduce-overhead')(img)",
    "torch.compile(su, mode='max-autotune')(img)",
    "torch.compile(su, dynamic=True)(img)",
    "torch.compile(su, dynamic=True, mode='reduce-overhead')(img)",
    "torch.compile(su, dynamic=True, mode='max-autotune')(img)",
]

In [5]:
compare = benchmark(stmts, tensors_fixed=False)
compare.print()

[------------------------------------ su ------------------------------------]
                                                                    |  runtime
1 threads: -------------------------------------------------------------------
      su(img)                                                       |  10426.5
      torch.compile(su)(img)                                        |   1333.6
      torch.compile(su, mode='reduce-overhead')(img)                |    858.8
      torch.compile(su, mode='max-autotune')(img)                   |    859.6
      torch.compile(su, dynamic=True)(img)                          |    859.7
      torch.compile(su, dynamic=True, mode='reduce-overhead')(img)  |    860.0
      torch.compile(su, dynamic=True, mode='max-autotune')(img)     |    860.0

Times are in microseconds (us).



The runtime measurements show that `torch.compile` enables a 12x speedup over the eager CUDA-based implementation. For reference, the CPU-based implementation takes roughly 3.5 seconds (= 3 548 992 us) on average, making the eager implementation approximately 340x and the `torch.compile` implementation 4125x faster.

As expected, the `reduce-overhead` (CUDA graphs) and `max-autotune` (CUDA graphs + Triton autotune) modes lead to further performance gains over the default `torch.compile`.